# Lab: Understanding ReAct (🤖 Reason + ⚡ Act) with OpenAI

Welcome! In this lab, we'll explore the **ReAct** prompting technique. ReAct is a powerful pattern that enables Large Language Models (LLMs) to solve complex problems by combining reasoning and action steps.

### What is ReAct?

Instead of just asking an LLM for a final answer, we ask it to "think out loud." It breaks down a problem into a sequence of steps:

1.  **Thought 🤔:** The model reasons about the problem and decides what to do next.
2.  **Action ⚡:** The model chooses a tool or action to perform to gather information.
3.  **Observation 👀:** The result of the action is given back to the model.

This loop repeats until the model has enough information to give a final answer. It makes the model's process more transparent and often more accurate, especially for questions that require up-to-date information or calculations.

---

## 1. Setup

First, let's install the necessary library and set up our OpenAI API key.

In [ ]:
%pip install -q openai python-dotenv

In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
from getpass import getpass

# Load environment variables
load_dotenv()

# It's best practice to use environment variables for API keys.
# If you don't have it set, this will prompt you to enter it securely.
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API Key: ')

client = OpenAI()

print("✅ OpenAI client initialized successfully!")

---

## 2. Standard Prompt vs. ReAct Prompt

Let's start with a simple question and see the difference between a standard prompt and a ReAct-style prompt.

**Question:** *"Who won the men's singles title at Wimbledon in 2023 and who did he beat in the final?"*

### Standard Prompt

Here, we just ask the question directly.

In [ ]:
standard_prompt = "Who won the men's singles title at Wimbledon in 2023 and who did he beat in the final?"

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": standard_prompt}],
    temperature=0
)

print(response.choices[0].message.content)

### ReAct-style Prompt

Now, let's instruct the model to use the ReAct format. We are not using external tools yet, just asking it to show its reasoning process.

In [ ]:
react_prompt = """
Answer the following question by reasoning step-by-step. Use the following format:

Thought: Your reasoning about the question.
Final Answer: The final answer to the user's question.

Question: Who won the men's singles title at Wimbledon in 2023 and who did he beat in the final?
"""

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": react_prompt}],
    temperature=0
)

print(response.choices[0].message.content)

**Observation:** Notice how the ReAct prompt forces the model to lay out its thinking process. This isn't super useful yet because the model is just using its internal knowledge. The real power comes when we give it **tools** to use.

---

## 3. ReAct with Tools (Manual Method)

Now for the fun part! We'll create a simulated environment where the LLM can use external tools. We will define two Python functions that our model can learn to call:

1.  `search(query)`: Simulates a web search.
2.  `calculate(expression)`: Simulates a calculator.

To handle all the new problems, we'll expand the hardcoded knowledge base in our `search` tool.

In [ ]:
# Expanded knowledge base for our simulated search engine
KNOWLEDGE_BASE = {
    "when was the first iphone released": "The first iPhone was released on June 29, 2007.",
    "us president in 2007": "George W. Bush was the US president in 2007. He was born on July 6, 1946.",
    "first iphone release date": "The first iPhone was released on June 29, 2007.",
    "george w. bush birthdate": "George W. Bush was born on July 6, 1946.",
    "us president june 29, 2007": "George W. Bush was the US president in 2007. He was born on July 6, 1946.",
    "when did man land on the moon": "The first moon landing was on July 20, 1969.",
    "uk monarch in 1969": "Queen Elizabeth II was the monarch of the United Kingdom in 1969. She was born on April 21, 1926.",
    "queen elizabeth ii birthdate": "Queen Elizabeth II was born on April 21, 1926.",
    "who composed the four seasons": "Antonio Vivaldi composed 'The Four Seasons'. He died in 1741.",
    "antonio vivaldi death year": "Antonio Vivaldi died in 1741.",
    "earth orbital speed": "The Earth's average orbital speed is approximately 107,000 kilometers per hour.",
    "capital of australia": "The capital of Australia is Canberra.",
    "population of canberra": "As of 2023, the population of Canberra is approximately 467,000.",
    "boiling point of water celsius": "The boiling point of water at sea level is 100 degrees Celsius.",
    "fifa world cup 2014 winner": "Germany won the FIFA World Cup in 2014. The runner-up was Argentina.",
    "main ingredient margarita cocktail": "The main ingredient in a Margarita cocktail is tequila."
}

def search(query: str) -> str:
    """Simulates searching the web for a given query."""
    print(f"\n🤖 SEARCHING for: '{query}'...")
    query = query.lower()
    return KNOWLEDGE_BASE.get(query, "Sorry, I couldn't find any information on that.")

def calculate(expression: str) -> str:
    """Simulates a calculator that can evaluate a simple math expression."""
    print(f"\n🧮 CALCULATING: '{expression}'...")
    try:
        # A safer way to evaluate a string expression
        result = eval(expression, {"__builtins__": {}}, {})
        return f"The result is {result}."
    except Exception as e:
        return f"Error: {e}"

# Test our tools
print(search("uk monarch in 1969"))
print(calculate("2024 - 1741"))

### The Manual ReAct Loop

Now, we'll solve a multi-step problem. We will act as the "computer" that executes the actions proposed by the LLM. We will feed the observations back to the model until it finds the final answer.

**Our complex question:** *"Who was the president of the United States when the first iPhone was released, and what was his age at that time?"*

In [ ]:
# This is the core prompt that tells the LLM how to behave
react_system_prompt = """
You are a helpful assistant that answers questions by breaking them down into a series of thoughts and actions. Your goal is to find the final answer.

You have access to the following tools:
1. search(query: str): Use this to find information about current events, facts, and people.
2. calculate(expression: str): Use this to perform mathematical calculations.

Follow this format strictly:
Thought: Your reasoning about what to do next to get closer to the answer.
Action: The tool you want to use, in the format `search("your query")` or `calculate("your expression")`.

After an action, you will receive an Observation. You will then repeat the Thought/Action cycle. Once you have enough information, provide the final answer in this format:
Final Answer: [Your conclusive answer here]
"""

In [ ]:
def run_manual_react_loop(question: str, max_steps=5):
    """Runs the manual ReAct loop for a given question."""
    messages = [
        {"role": "system", "content": react_system_prompt},
        {"role": "user", "content": question}
    ]
    
    for i in range(max_steps):
        print(f"\n--- STEP {i+1} ---")

        print("🤔 Thinking...")
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            temperature=0,
            stop=["\nObservation:"] # Stop generation when it's time for us to provide an observation
        )
        llm_output = response.choices[0].message.content
        print(llm_output)

        messages.append({"role": "assistant", "content": llm_output})

        if "Final Answer:" in llm_output:
            print("\n✅ ReAct loop finished!")
            break

        action_str = llm_output.split("Action:")[-1].strip()
        observation = ""
        try:
            if action_str.startswith("search("):
                query = action_str[len("search("):-1].strip('"')
                observation = search(query)
            elif action_str.startswith("calculate("):
                expression = action_str[len("calculate("):-1].strip('"')
                observation = calculate(expression)
            else:
                observation = "Error: Invalid action specified."
        except Exception as e:
             observation = f"Error executing action: {e}"

        print(f"👀 OBSERVATION: {observation}")
        messages.append({"role": "user", "content": f"Observation: {observation}"})
    else:
        print("\n⚠️ ReAct loop reached max steps.")

# Let's run it with the original question!
original_question = "Who was the president of the United States when the first iPhone was released, and what was his age at that time?"
run_manual_react_loop(original_question)

---

## 4. Advanced ReAct: Automated Tool Calling

The manual method is great for understanding the ReAct concept, but it's brittle. It relies on the model generating text in a *perfect* format that we have to parse. 

Modern LLM APIs (like OpenAI's) have a much better way: **structured tool calling**. 

Here's how it works:
1.  **Define Tools:** We describe our Python functions to the model in a structured format (JSON Schema).
2.  **API Call:** We send the conversation history and the list of tools to the model.
3.  **Model Response:** If the model decides to use a tool, it doesn't just write text. It returns a `tool_calls` object containing the exact function name and arguments it wants to use.
4.  **Execute & Respond:** Our code executes the specified function with the given arguments and sends the result back to the model in a new turn. 

This is more reliable, secure, and the standard for building modern AI agents.

In [ ]:
# 1. Define tools in the format the OpenAI API expects.
tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "Searches the web for information on various topics, including dates, facts, and people.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query (e.g., 'when did man land on the moon')"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Performs mathematical calculations.",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "The mathematical expression to evaluate (e.g., '1969 - 1926')"
                    }
                },
                "required": ["expression"]
            }
        }
    }
]

# Map tool names to actual functions
available_functions = {
    "search": search,
    "calculate": calculate,
}

In [ ]:
def run_tool_calling_react_loop(question: str, max_steps=5):
    """Runs the ReAct loop for a given question using structured tool calling."""
    messages = [{"role": "user", "content": question}]
    
    for i in range(max_steps):
        print(f"\n--- STEP {i+1} ---")
        print("🤔 Thinking...")

        # The core API call, now with tools!
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            tools=tools,
            tool_choice="auto", # The model decides when to call tools
        )

        response_message = response.choices[0].message
        tool_calls = response_message.tool_calls

        # 2. Check if the model wants to call a tool
        if tool_calls:
            print(f"⚡ Action: Decided to call {len(tool_calls)} tool(s).")
            messages.append(response_message) # Append the assistant's message with tool calls

            # 3. Execute the tools
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                
                # Call the actual Python function
                function_response = function_to_call(**function_args)
                
                print(f"👀 OBSERVATION (from {function_name}): {function_response}")
                
                # 4. Append the tool's output to the conversation
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )
        else:
            # No tool call, so it must be the final answer
            final_answer = response_message.content
            print(f"\n✅ Final Answer: {final_answer}")
            break
    else:
        print("\n⚠️ ReAct loop reached max steps.")

### Tool Calling - Problem 1

**Question:** *"Who was the monarch of the United Kingdom when the first person landed on the moon, and how old were they at that time?"*

In [ ]:
problem_1 = "Who was the monarch of the United Kingdom when the first person landed on the moon, and how old were they at that time?"
run_tool_calling_react_loop(problem_1)

### Tool Calling - Problem 2

**Question:** *"Who composed 'The Four Seasons', and roughly how many years passed between his death and the release of the first iPhone?"*

In [ ]:
problem_2 = "Who composed 'The Four Seasons', and roughly how many years passed between his death and the release of the first iPhone?"
run_tool_calling_react_loop(problem_2, max_steps=6) # Allow more steps if needed

### Tool Calling - Problem 3

**Question:** *"What is the Earth's average orbital speed in kilometers per hour?"*

In [ ]:
problem_3 = "What is the Earth's average orbital speed in kilometers per hour?"
run_tool_calling_react_loop(problem_3)

---

## 5. Additional Practice (Manual ReAct Loop)

Now, try solving these remaining problems using the **original manual ReAct loop** (`run_manual_react_loop`). This will help reinforce the fundamental `Thought -> Action -> Observation` cycle.

### Manual Loop - Problem 4

**Question:** *"What is the capital of Australia and what is its population?"*

In [ ]:
problem_4 = "What is the capital of Australia and what is its population?"
run_manual_react_loop(problem_4)

### Manual Loop - Problem 5

**Question:** *"How do I convert the boiling point of water from Celsius to Fahrenheit? The formula is (C * 9/5) + 32."*

In [ ]:
problem_5 = "First, find the boiling point of water in Celsius. Then, calculate the equivalent in Fahrenheit using the formula (C * 9/5) + 32."
run_manual_react_loop(problem_5)

### Further Challenges

Here are more questions you can try to solve on your own by calling the `run_manual_react_loop` function. 

6. **FIFA World Cup:** *"Who won the FIFA World Cup in 2014, and who was the runner-up?"*
7. **Movie Time:** *"If a movie is 148 minutes long, how long is it in hours and minutes?"* (Hint: this requires calculation, specifically division and modulo)
8. **Simple Interest:** *"If I invest $5,000 at an annual interest rate of 7% for 10 years (simple interest), what will my total interest earnings be?"* (Formula: Principal * Rate * Time)
9. **Cocktails:** *"What is the main ingredient in a Margarita cocktail?"*
10. **Literature:** *"Who wrote 'The Hitchhiker's Guide to the Galaxy' and when was it first published?"* (Note: You may need to add this info to the `KNOWLEDGE_BASE`)

---

## 6. Conclusion & Key Takeaways

Congratulations on completing the expanded ReAct lab! 🥳

You've successfully guided an LLM to solve complex problems using both a manual ReAct loop and the more modern, structured tool-calling approach.

### Key Takeaways:

1.  **Transparency:** The `Thought` process shows you *how* the model is working, making it easier to debug and trust.
2.  **Accuracy:** By using external tools (like search or a calculator), the model can access up-to-date information and perform precise calculations, overcoming its inherent limitations.
3.  **Extensibility:** You can create any tool you want! Imagine giving the model tools to check your calendar, send an email, or query a database.
4.  **Evolution of ReAct:** You've seen the progression from parsing raw text (the manual loop) to using structured **tool calling**. The latter is far more robust and is the standard method used in modern agentic frameworks.

**In the real world**, frameworks like **LangChain**, **LlamaIndex**, and direct SDKs for models like GPT-4o automate the tool-calling loop, making it much easier to build powerful, tool-augmented AI applications. Understanding the fundamental `Thought -> Action -> Observation` cycle is the key to mastering them.